# FUSION FINALE DES JEUX DE DONNEES

In [1]:
from utils.io import dataframe_viewer, files_search, data_merger, data_validation, data_overview, \
gen_id_from_ech

import re, os
import pandas as pd
from definitions import ROOT_DIR

In [2]:
def create_df(files, verbose=True): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    files: list of files name
    """
    dfs = []
    i = 0
    for f in files:
        i += 1
        df = pd.read_csv(f, delimiter=',')
        dfs.append(df)
        
        if verbose:
            if 'X' in list(df.columns): msg = ' --> Coordinates'
            else: msg = ' --> No coordinates'

            print(f"df{i} : {msg}")
            
    return dfs

In [3]:
csv_data_dir = ROOT_DIR + '/CF_data/Result_traitem/organisation/'
save_dir = csv_data_dir + '../fusion_finale/'

In [4]:
os.system(f"rm -rf {save_dir}") 
os.system(f"mkdir {save_dir}") 

0

In [5]:
sufx = ['sup', 'prof', 'inf', '/\dM(\*)?']
prefx = ['eau forage ']
id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*'

# Collecte des fichiers

In [6]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}
data_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}

In [7]:
files_search(csv_data_dir, files_dict, prefix='source', details=False)

Borehole  	:  14
Litho  	:  4
Equipm  	:  6
Measure  	:  13
Sample  	:  13
Unknow  	:  1


In [8]:
how=['inner', 'outer', 'left', 'right']
view = False

## Forages

In [9]:
key='Borehole'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

14 files


In [10]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Forage_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Prof_contact_sol_forage/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSD

In [11]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 4, 8, 9, 11, 13]
Files without coordinates:[0, 5, 6, 7, 10, 12]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [12]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Boreholes.csv' # 1
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Boreholes.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 57, columns : 8, Unique values on cols: {'ID': 57}
Rows : 14, columns : 7, Unique values on cols: {'ID': 14}


(None, None)

In [13]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [14]:
dataset = mdf.copy()

In [15]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 69


#### $\color{green}{\textbf{Lecture et fusion}}$

In [16]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Boreholes.csv' # 3
file2= csv_data_dir + 'Prof_contact_sol_forage/source_merge/source_Boreholes.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 22, columns : 8, Unique values on cols: {'ID': 22}
Rows : 8, columns : 6, Unique values on cols: {'ID': 8}


(None, None)

In [17]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [18]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

In [19]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 77


#### $\color{green}{\textbf{Lecture et fusion}}$

In [20]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Boreholes.csv' # 8
file2= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Boreholes.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 520, columns : 20, Unique values on cols: {'ID': 518}
Rows : 25, columns : 17, Unique values on cols: {'ID': 25}


(None, None)

In [21]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [22]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [23]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Type_y':list(conflict_df.index), 'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [24]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 605


#### $\color{green}{\textbf{Lecture et fusion}}$

In [25]:
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Boreholes.csv' # 11
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Boreholes.csv' # 13

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 27, columns : 19, Unique values on cols: {'ID': 27}
Rows : 78, columns : 9, Unique values on cols: {'ID': 77}


(None, None)

In [26]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [27]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [28]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Date_for_y':list(conflict_df.index), 'Type_y':list(conflict_df.index), 
                            'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [29]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 677


#### $\color{green}{\textbf{Lecture et fusion}}$

In [30]:
file1= csv_data_dir + 'Forage_Pilote/source_merge/source_Boreholes.csv' # 0
file2= csv_data_dir + 'Siterem_Ext_Pilote/source_merge/source_Boreholes.csv' # 5

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 72, columns : 5, Unique values on cols: {'ID': 72}
Rows : 10, columns : 5, Unique values on cols: {'ID': 6}


(None, None)

In [31]:
df2['ID'] = df2['ID'].astype('object')

In [32]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [33]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [34]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Z_x':list(conflict_df.index), 'Type_x':list(conflict_df.index), 
                            'Long_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [35]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 701


#### $\color{green}{\textbf{Lecture et fusion}}$

In [36]:
file1= csv_data_dir + 'Siterem_Pilote/source_merge/source_Boreholes.csv' # 6
file2= csv_data_dir + 'Siterem_Result_Sol/source_merge/source_Boreholes.csv' # 7

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 22, columns : 4, Unique values on cols: {'ID': 16}
Rows : 23, columns : 4, Unique values on cols: {'ID': 23}


(None, None)

In [37]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [38]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [39]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_pz_x':list(conflict_df.index), 'Type_x':list(conflict_df.index), 
                            'Long_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [40]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 714


#### $\color{green}{\textbf{Lecture et fusion}}$

In [41]:
file1= csv_data_dir + 'observ_terrain/source_merge/source_Boreholes.csv' # 10
file2= csv_data_dir + 'result_sol_ext_pilote/source_merge/source_Boreholes.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 27, columns : 6, Unique values on cols: {'ID': 27}
Rows : 19, columns : 4, Unique values on cols: {'ID': 19}


(None, None)

In [42]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID'], dist_max=1)

In [43]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [44]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_pz_sol_x':[18], 'Long_pz_sol_y':[i for i in conflict_df.index if i not in [18]], 
                            'Rmq_y':list(conflict_df.index), 'Type_x':list(conflict_df.index),})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [45]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 717


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [46]:
data_dict['Borehole'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 717, columns : 30, Unique values on cols: {'ID': 704, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=717, min=3, readout=False), IntSlider(value=1…

In [47]:
dataset.to_csv(save_dir + 'Boreholes.csv', index=False)

###  ====================================================

# Lithologies

In [48]:
key='Litho'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

4 files


In [49]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/donnees_terrain_2019/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/profils_sols_donnees_forages/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/vUmons_logsFor/source_merge/source_Lithologies.csv']

In [50]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[0, 2, 3]
Files without coordinates:[1]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [51]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Lithologies.csv' # 0
file2= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Lithologies.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 1627, columns : 9, Unique values on cols: {'ID': 298}
Rows : 55, columns : 9, Unique values on cols: {'ID': 25}


(None, None)

In [52]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [53]:
dataset = mdf.copy()

In [54]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1682


#### $\color{green}{\textbf{Lecture et fusion}}$

In [55]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Lithologies.csv' # 1
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Lithologies.csv' # 3

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> Coordinates
Rows : 93, columns : 9, Unique values on cols: {'ID': 25}
Rows : 112, columns : 9, Unique values on cols: {'ID': 71}


(None, None)

In [56]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [57]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [58]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1887


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [59]:
data_dict['Litho'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 1887, columns : 12, Unique values on cols: {'ID': 418, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=1887, min=3, readout=False), IntSlider(value=…

In [60]:
dataset.to_csv(save_dir + 'Lithologies.csv', index=False)

###  ====================================================

# Echantillons

In [12]:
key='Sample'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

13 files


In [13]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Container_phyto/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Liste_XY/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/org

In [14]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 4, 8, 10, 12]
Files without coordinates:[0, 5, 6, 7, 9, 11]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [64]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Samples.csv' # 2
file2= csv_data_dir + 'Liste_XY/source_merge/source_Samples.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 105, columns : 91, Unique values on cols: {'ID': 60}
Rows : 254, columns : 6, Unique values on cols: {'ID': 254}


(None, None)

In [65]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech'], dist_max=1)

In [66]:
dataset = mdf.copy()

In [67]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 358


#### $\color{green}{\textbf{Lecture et fusion}}$

In [68]:
file1= csv_data_dir +  'Phase_1_Memoris/source_merge/source_Samples.csv' # 3
file2= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Samples.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 46, columns : 84, Unique values on cols: {'ID': 29}
Rows : 36, columns : 83, Unique values on cols: {'ID': 22}


(None, None)

In [69]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 Date_ech
1 ID_ech
1 ID


In [70]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [71]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [72]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_y':list(conflict_df.index), 'C21-C35_y':list(conflict_df.index), 
                            'C12-C16_y':list(conflict_df.index), 'Fract_2+_y':list(conflict_df.index), 
                            'C10-C12_y':list(conflict_df.index), 'HC_tot_C10-C35_y':list(conflict_df.index), 
                            'Fract_2_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [73]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 404


#### $\color{green}{\textbf{Lecture et fusion}}$

In [74]:
file1= csv_data_dir +  'database_Memoris3/source_merge/source_Samples.csv' # 8
file2= csv_data_dir +  'profils_sols_donnees_forages/source_merge/source_Samples.csv' # 10

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 1583, columns : 197, Unique values on cols: {'ID': 470}
Rows : 41, columns : 9, Unique values on cols: {'ID': 27}


(None, None)

In [75]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_for
0 Date_ech
0 ID
0 ID_ech
1 Date_for
1 ID
1 ID_ech


In [76]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech'], dist_max=1)

In [77]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [78]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Nappe_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [79]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1977


#### $\color{green}{\textbf{Lecture et fusion}}$

In [80]:
file1= csv_data_dir +  'Container_phyto/source_merge/source_Samples.csv' # 0
file2= csv_data_dir +  'vUmons_logsFor/source_merge/source_Samples.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> Coordinates
Rows : 6, columns : 82, Unique values on cols: {'ID': 'NA'}
Rows : 45, columns : 76, Unique values on cols: {'ID': 29}


(None, None)

In [81]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
1 Date_ech
1 ID_ech
1 ID


In [82]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID_ech', 'Date_ech'], dist_max=1)

In [83]:
mdf = gen_id_from_ech(mdf, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [84]:
data = mdf
col = 'ID_ech'
for i in data.index:
    v = data.loc[i, col]
    if not pd.isnull(v) and re.search('ech', v, re.I):
        data.loc[i, 'ID'] = 'F_' + re.sub(' |.','', v,re.I)
mdf = data.copy()

In [85]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [86]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2028


#### $\color{green}{\textbf{Lecture et fusion}}$

In [87]:
file1= csv_data_dir +  'Siterem_Ext_Pilote/source_merge/source_Samples.csv' # 5
file2= csv_data_dir +  'Siterem_Pilote/source_merge/source_Samples.csv' # 6

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 44, columns : 97, Unique values on cols: {'ID': 7}
Rows : 95, columns : 97, Unique values on cols: {'ID': 12}


(None, None)

In [88]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 Date_ech
1 ID_ech
1 ID


In [89]:
df1['ID_ech'] = df1['ID_ech'].astype('object')

In [90]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID_ech', 'Date_ech'], dist_max=1)

In [91]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [92]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_x':list(conflict_df.index), 'Type_ech_x':list(conflict_df.index), 
                            'C12-C16_x':list(conflict_df.index), 'C10-C12_x':list(conflict_df.index), 
                            'HC_tot_C10-C35_x':list(conflict_df.index),  'C21-C35_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [93]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2165


#### $\color{green}{\textbf{Lecture et fusion}}$

In [94]:
file1= csv_data_dir +  'Siterem_Result_Sol/source_merge/source_Samples.csv' # 7
file2= csv_data_dir +  'donnees_terrain_2019/source_merge/source_Samples.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 60, columns : 82, Unique values on cols: {'ID': 23}
Rows : 70, columns : 9, Unique values on cols: {'ID': 22}


(None, None)

In [95]:
df2['ID'] = df2['ID'].astype('object')

In [96]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 ID
1 ID_ech


In [97]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech'], dist_max=1)

In [98]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2294


#### $\color{green}{\textbf{Lecture et fusion}}$

In [100]:
file1= csv_data_dir +  'result_sol_ext_pilote/source_merge/source_Samples.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
Rows : 53, columns : 85, Unique values on cols: {'ID': 19}


In [101]:
df_list = [df1]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID
0 ID_ech


In [102]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [103]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2308


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [104]:
data_dict['Sample'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2308, columns : 219, Unique values on cols: {'ID': 819, 'ID_ech': 1878}


interactive(children=(IntSlider(value=3, description='rows', max=2308, min=3, readout=False), IntSlider(value=…

In [105]:
dataset.to_csv(save_dir + 'Samples.csv', index=False)

###  ====================================================

## Equipements

In [15]:
key='Equipm'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

6 files


In [16]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/donnees_terrain_2019/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/profils_sols_donnees_forages/source_merge/source_Equipments.csv']

In [17]:
data_overview(files_dict[key])

EmptyDataError: No columns to parse from file

#### $\color{green}{\textbf{Lecture et fusion}}$

In [64]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Samples.csv' # 2
file2= csv_data_dir + 'Liste_XY/source_merge/source_Samples.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 105, columns : 91, Unique values on cols: {'ID': 60}
Rows : 254, columns : 6, Unique values on cols: {'ID': 254}


(None, None)

In [65]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech'], dist_max=1)

In [66]:
dataset = mdf.copy()

In [67]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 358


#### $\color{green}{\textbf{Lecture et fusion}}$

In [68]:
file1= csv_data_dir +  'Phase_1_Memoris/source_merge/source_Samples.csv' # 3
file2= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Samples.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 46, columns : 84, Unique values on cols: {'ID': 29}
Rows : 36, columns : 83, Unique values on cols: {'ID': 22}


(None, None)

In [69]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 Date_ech
1 ID_ech
1 ID


In [70]:
mdf, conflict_df=data_merger(df1, df2, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [71]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how=how[1], on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [72]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_y':list(conflict_df.index), 'C21-C35_y':list(conflict_df.index), 
                            'C12-C16_y':list(conflict_df.index), 'Fract_2+_y':list(conflict_df.index), 
                            'C10-C12_y':list(conflict_df.index), 'HC_tot_C10-C35_y':list(conflict_df.index), 
                            'Fract_2_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [73]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 404


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [104]:
data_dict['Equipm'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2308, columns : 219, Unique values on cols: {'ID': 819, 'ID_ech': 1878}


interactive(children=(IntSlider(value=3, description='rows', max=2308, min=3, readout=False), IntSlider(value=…

In [105]:
dataset.to_csv(save_dir + 'Equipments.csv', index=False)

###  ====================================================

## Objets inconnus

In [108]:
key='Unknow'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

1 files


In [109]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Unknown.csv']

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [104]:
data_dict['Unknow'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2308, columns : 219, Unique values on cols: {'ID': 819, 'ID_ech': 1878}


interactive(children=(IntSlider(value=3, description='rows', max=2308, min=3, readout=False), IntSlider(value=…

In [105]:
dataset.to_csv(save_dir + 'Unknown.csv', index=False)

###  ====================================================

###  ====================================================

# Mesures

In [110]:
key='Measure'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

13 files


In [111]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Container_phyto/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/R

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [104]:
data_dict['Measure'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2308, columns : 219, Unique values on cols: {'ID': 819, 'ID_ech': 1878}


interactive(children=(IntSlider(value=3, description='rows', max=2308, min=3, readout=False), IntSlider(value=…

In [105]:
dataset.to_csv(save_dir + 'Measures.csv', index=False)

###  ====================================================

# Croisement entre jeux de données

## Querying